In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init as init

In [2]:
import visdom

vis = visdom.Visdom()
vis.close(env='main')

Setting up a new session...


''

In [3]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num, Y=loss_value, win=loss_plot, update='append')

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.set_device(1)
    torch.cuda.manual_seed_all(777)
    
    device_name = torch.cuda.get_device_name()
    print(device_name, 'is ready.')

In [5]:
learning_rate = .001
training_epochs = 20
batch_size = 512

In [6]:
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test  = dsets.MNIST(root='MNIST_data/', train=False, transform=transforms.ToTensor(), download=True)

In [7]:
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [8]:
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, ceil_mode=True)
        )
        

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(4*4*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10)
        
        init.xavier_uniform_(self.fc1.weight)
        init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.flatten(out)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out

In [9]:
model = CNN_Model().to(device)

In [10]:
import torchsummaryX

torchsummaryX.summary(model, torch.zeros(1, 1, 28, 28))

                         Kernel Shape     Output Shape     Params  Mult-Adds
Layer                                                                       
0_layer1.Conv2d_0       [1, 32, 3, 3]  [1, 32, 28, 28]      320.0   225.792k
1_layer1.ReLU_1                     -  [1, 32, 28, 28]          -          -
2_layer1.MaxPool2d_2                -  [1, 32, 14, 14]          -          -
3_layer2.Conv2d_0      [32, 64, 3, 3]  [1, 64, 14, 14]    18.496k  3.612672M
4_layer2.ReLU_1                     -  [1, 64, 14, 14]          -          -
5_layer2.MaxPool2d_2                -    [1, 64, 7, 7]          -          -
6_layer3.Conv2d_0     [64, 128, 3, 3]   [1, 128, 7, 7]    73.856k  3.612672M
7_layer3.ReLU_1                     -   [1, 128, 7, 7]          -          -
8_layer3.MaxPool2d_2                -   [1, 128, 4, 4]          -          -
9_flatten                           -        [1, 2048]          -          -
10_fc1                    [2048, 625]         [1, 625]  1.280625M      1.28M

In [13]:
import torchsummary

torchsummary.summary(model, (1, 28, 28), batch_size=1, device='cpu')

---------------------------------------------------------------------------------------------------------------------------------------
        Layer (type)    Kernel Shape          Input Shape                        Output Shape              Param #          Multi Ops #
            Conv2d-1   [1, 32, 3, 3]       [1, 1, 28, 28]                     [1, 32, 28, 28]              225,824              225,792
              ReLU-2               -      [1, 32, 28, 28]                     [1, 32, 28, 28]                    0                    0
         MaxPool2d-3               -      [1, 32, 28, 28]                     [1, 32, 14, 14]                    0                    0
            Conv2d-4  [32, 64, 3, 3]      [1, 32, 14, 14]                     [1, 64, 14, 14]            3,612,736            3,612,672
              ReLU-5               -      [1, 64, 14, 14]                     [1, 64, 14, 14]                    0                    0
         MaxPool2d-6               -      [1, 64

(8738245, 8738245)

In [12]:
print(model)

value = torch.Tensor(1, 1, 28, 28).cuda()
print(model(value).shape)

CNN_Model(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (flatten): Flatten()
  (fc1): Linear(in_features=2048, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)


AssertionError: Torch not compiled with CUDA enabled

In [11]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title='Tracking Losses', legend=['Loss'], showlegend=True))

In [12]:
from IPython.display import clear_output

def train(model):
    model.train()
    total_batch = len(data_loader)

    import time
    s = time.time()
    for epoch in range(training_epochs):
        avg_cost = 0

        for x, y in data_loader:
            x = x.to(device)
            y = y.to(device)

            hypothesis = model(x)
            cost = criterion(hypothesis, y)
            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            avg_cost += cost / total_batch

        clear_output(wait=True)
        print("[Epoch: {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost))
        loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))

    print("Learning Finished")
    print("Time Taken {:.6f}s by {}".format(time.time()-s, device))

In [13]:
%timeit -n 1 -r 1 train(model)

[Epoch:   20] cost = 0.00881488714
Learning Finished
Time Taken 102.720498s by cuda
1min 42s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [14]:
print(mnist_test.data.shape)

torch.Size([10000, 28, 28])


In [15]:
def test(model):
    model.eval()
    
    with torch.no_grad():
        x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
        y_test = mnist_test.test_labels.to(device)
        
        prediction = model(x_test)
        correct_prediction = torch.argmax(prediction, 1) == y_test
        acc =  correct_prediction.float().mean()
        
        print("Accuracy: {:.2f}%".format(acc.item()))
        torch.cuda.empty_cache()

In [17]:
test(model)

Accuracy: 0.99%
